In [ ]:
! pip install transformers peft evaluate
! pip install --upgrade datasets

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import os
import torch
import evaluate
import torch.nn as nn
import argparse
from tqdm import tqdm
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification,get_linear_schedule_with_warmup,set_seed
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftType, PrefixTuningConfig, PromptEncoderConfig, PromptTuningConfig

In [4]:
batch_size=32
model_name='roberta-large'
task='mrpc'
peft_type=PeftType.PROMPT_TUNING
device='cuda'
num_epochs=20

In [5]:
# We adding training Parameter (prompt)
peft_config=PromptTuningConfig(task_type='SEQ_CLS',num_virtual_tokens=10)
lr=1e-3

In [7]:
# This will add padding according to LLM
if any(k in model_name for k in ('gpt','opt','bloom')):
  padding_side='left'
else:
  padding_side='right'

tokenizer=AutoTokenizer.from_pretrained(model_name,padding_side=padding_side)
# Add padding to End of sentence
if getattr(tokenizer, 'pad_token_id') is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Load Dataset
datasets=load_dataset('glue',task)
metric=evaluate.load('glue',task)

# Tokenize the Data
def tokenize_function(examples):
  outputs=tokenizer(examples['sentence1'],examples['sentence2'],truncation=True,max_length=None)
  return outputs

# Apply func
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['idx','sentence1','sentence2']
)

# Rename Label
tokenized_datasets=tokenized_datasets.rename_column('label','labels')


# Combine them into dictonary tokenized inputs
def collate_fn(examples):
  return tokenizer.pad(examples, padding='longest',return_tensors='pt')

# Initiate Dataloader
train_dataloader = DataLoader(tokenized_datasets['train'],shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets['validation'],shuffle=False, collate_fn=collate_fn,batch_size=batch_size)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:
# Load pretrained base model
model=AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True)
# Wrap base model with PEFT
model=get_peft_model(model, peft_config)
# Trainable parameters after PEFT
model.print_trainable_parameters()
model

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 10,240 || all params: 355,372,034 || trainable%: 0.0029


PeftModelForSequenceClassification(
  (base_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): Robe

In [9]:
# Optimizer
optimizer= AdamW(params=model.parameters(),lr=lr)

# Learning Rate- Linearly increase the LR
lr_scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps = 0.06 * (len(train_dataloader)* num_epochs), # Number of steps to warmup the LR
    num_training_steps= (len(train_dataloader)* num_epochs)    # Total steps for training
)

In [10]:
model.to(device)
# Training model on batch data
for epoch in range(num_epochs):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        # Compute gradients via backpropagation
        loss.backward()
        # Update model weights
        optimizer.step()
        # Adjust Learning rate as scheduled
        lr_scheduler.step()
        # Clear old gradients
        optimizer.zero_grad()
  # Evaluating after each epoch
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    # Runs the model without tracking gradient
    with torch.no_grad():
      outputs=model(**batch)
      # Get the predictions
    predictions= outputs.logits.argmax(dim=-1)
    predictions, references =predictions, batch['labels']
    metric.add_batch(
        predictions=predictions,
        references=references
    )
  # Compute evaluation metric
  eval_metric = metric.compute()
  print(f"Epoch {epoch}:",eval_metric)


100%|██████████| 13/13 [00:06<00:00,  1.90it/s]


Epoch 0: {'accuracy': 0.625, 'f1': 0.7445742904841403}


100%|██████████| 13/13 [00:06<00:00,  1.93it/s]


Epoch 1: {'accuracy': 0.6348039215686274, 'f1': 0.7470288624787776}


100%|██████████| 13/13 [00:06<00:00,  1.91it/s]


...

Epoch 17: {'accuracy': 0.6470588235294118, 'f1': 0.7639344262295082}


100%|██████████| 13/13 [00:06<00:00,  1.92it/s]


Epoch 18: {'accuracy': 0.6397058823529411, 'f1': 0.7554076539101497}


100%|██████████| 13/13 [00:06<00:00,  1.90it/s]

Epoch 19: {'accuracy': 0.6421568627450981, 'f1': 0.7566666666666667}


In [11]:
# Push model to HUB
model.push_to_hub('OmBhandwalkar/roberta-large-peft-prompt-tuning',use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:920: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/roberta-large-peft-prompt-tuning/commit/403346ae499dfc03166908ba684f8ce7da253bc1', commit_message='Upload model', commit_description='', oid='403346ae499dfc03166908ba684f8ce7da253bc1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/roberta-large-peft-prompt-tuning', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/roberta-large-peft-prompt-tuning'), pr_revision=None, pr_num=None)

In [12]:
# Load Adapter from Hub
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "OmBhandwalkar/roberta-large-peft-prompt-tuning"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

# Inference
inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
  # Forward pass without gradient
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

adapter_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

100%|██████████| 13/13 [00:06<00:00,  2.10it/s]

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}
